In [ ]:
# This notebook demonstrates the AI pipeline for face personalization
# We'll use a combination of face detection, style transfer, and template integration

import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN
import replicate
import os
from dotenv import load_dotenv

load_dotenv()

class FacePersonalizationPipeline:
    def __init__(self):
        # Initialize face detector
        self.face_detector = MTCNN(keep_all=True)
        
        # Set up device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Replicate API token
        self.replicate_token = os.getenv("REPLICATE_API_TOKEN")
    
    def detect_and_extract_face(self, image_path):
        """Detect and extract face from image"""
        # Load image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        boxes, probs = self.face_detector.detect(image_rgb)
        
        if boxes is None:
            raise ValueError("No face detected in the image")
        
        # Get the largest face
        face_areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in boxes]
        largest_idx = np.argmax(face_areas)
        box = boxes[largest_idx]
        
        # Extract face with padding
        padding = 50
        x1, y1, x2, y2 = box
        x1 = max(0, int(x1) - padding)
        y1 = max(0, int(y1) - padding)
        x2 = min(image.shape[1], int(x2) + padding)
        y2 = min(image.shape[0], int(y2) + padding)
        
        face_image = image[y1:y2, x1:x2]
        
        return face_image, (x1, y1, x2, y2)
    
    def apply_style_transfer(self, face_image, style="storybook"):
        """Apply style transfer using Replicate API"""
        # Save face image temporarily
        temp_path = "temp_face.png"
        cv2.imwrite(temp_path, face_image)
        
        # Prepare prompt based on style
        style_prompts = {
            "storybook": "cartoon illustration of a child's face, storybook style, colorful, detailed, magical, fantasy",
            "disney": "disney animation style, pixar character, 3D render, smooth lighting, cinematic",
            "watercolor": "watercolor painting, soft edges, artistic, brush strokes, pastel colors",
            "cartoon": "cartoon character, bold outlines, vibrant colors, comic book style"
        }
        
        prompt = style_prompts.get(style, style_prompts["storybook"])
        
        # Use Replicate API
        output = replicate.run(
            "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
            input={
                "prompt": prompt,
                "image": open(temp_path, "rb"),
                "num_outputs": 1,
                "guidance_scale": 7.5,
                "num_inference_steps": 30
            }
        )
        
        # Clean up temp file
        os.remove(temp_path)
        
        return output[0] if output else None
    
    def integrate_with_template(self, styled_face_url, template_path, face_position):
        """Integrate styled face into template"""
        # Download styled face
        import requests
        from io import BytesIO
        
        response = requests.get(styled_face_url)
        styled_face = Image.open(BytesIO(response.content))
        
        # Load template
        template = Image.open(template_path)
        
        # Resize face to fit template position
        x1, y1, x2, y2 = face_position
        face_width = x2 - x1
        face_height = y2 - y1
        
        # Resize styled face to match original face dimensions
        styled_face_resized = styled_face.resize((face_width, face_height))
        
        # Create result image
        result = template.copy()
        
        # Paste styled face onto result
        result.paste(styled_face_resized, (x1, y1))
        
        return result
    
    def process_image(self, image_path, template_path, style="storybook"):
        """Complete pipeline"""
        # Step 1: Face detection and extraction
        print("Step 1: Detecting face...")
        face_image, face_position = self.detect_and_extract_face(image_path)
        
        # Step 2: Style transfer
        print("Step 2: Applying style transfer...")
        styled_face_url = self.apply_style_transfer(face_image, style)
        
        if not styled_face_url:
            raise ValueError("Style transfer failed")
        
        # Step 3: Template integration
        print("Step 3: Integrating with template...")
        result = self.integrate_with_template(styled_face_url, template_path, face_position)
        
        return result

# Usage example
if __name__ == "__main__":
    pipeline = FacePersonalizationPipeline()
    
    # Process an image
    result = pipeline.process_image(
        image_path="child_photo.jpg",
        template_path="storybook_template.png",
        style="storybook"
    )
    
    # Save result
    result.save("personalized_storybook.png")
    print("Personalized illustration saved!")